In [1]:
!python -m pip install gradio

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.


In [2]:
from pathlib import Path
from PIL import Image

import glob
import os
import torch
import torchvision

import gradio as gr
import onnxruntime as ort
import torch.nn.functional as F
import torchvision.transforms as T

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def load_model(model, path):
    
    ckpt_dict = torch.load(path)
    
    model.load_state_dict(ckpt_dict['model_state_dict'])
    
    return ckpt_dict

In [4]:
path = '/workspace/models/ResNet18_7classes_cpu_83.52_1688138167.pt'

In [5]:
model = torch.load(path)

In [15]:
model = model.cpu()

In [16]:
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [17]:
def predict(inp):

    test_transforms = T.Compose(
        [
            T.Resize(size=(224, 224)),
            T.ToTensor(),
            T.Normalize(mean=torch.tensor([0.485, 0.456, 0.406]),
                        std=torch.tensor([0.229, 0.224, 0.225])),
    ])
    
    inp = test_transforms(inp).unsqueeze(0)
    
    class_list = [
        'beebalm',
        'dandelion',
        'juniper',
        'mullen',
        'plantain',
        'raspberry',
        'wild_rose_bush'
    ]
    
    with torch.no_grad():
        pred = F.softmax(model(inp)[0], dim=0)
        confidences = {class_list[i]: float(pred[i]) for i in range(7)}

    return confidences

In [18]:
gr.Interface(fn=predict,
             inputs=gr.Image(type="pil"),
             outputs=gr.Label(num_top_classes=3),
            ).launch(share=True)

Running on local URL:  http://127.0.0.1:7862
Running on public URL: https://933776ab9a96f343a2.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
